# Zusammenführen der einzelnen XML-Dateien

In [ ]:
import os
from lxml import etree

# Funktion, um TEI-Dateien zu filtern und zusammenzuführen
def merge_tei_files(input_folder, output_file, library_city):
    # Namespace für TEI-Dokumente definieren
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Ein neues Wurzelelement <TEI> für die kombinierte Datei erstellen
    combined_root = etree.Element("TEI", nsmap={None: "http://www.tei-c.org/ns/1.0"})

    # Alle Dateien im Eingabeordner durchlaufen
    for filename in os.listdir(input_folder):
        # Nur XML-Dateien berücksichtigen
        if filename.endswith(".xml"):
            file_path = os.path.join(input_folder, filename)

            # Die aktuelle TEI-Datei einlesen
            tree = etree.parse(file_path)
            root = tree.getroot()

            # Das <institution>-Element innerhalb von <msIdentifier> finden
            institution = root.find(".//tei:msIdentifier/tei:institution", namespaces)

            # Prüfen, ob das <institution>-Element existiert und die gewünschte Bibliothek enthält
            if institution is not None and library_city in institution.text:
                # Das <teiHeader>-Element aus der aktuellen Datei finden
                header_element = root.find(".//tei:teiHeader", namespaces)

                # Das <text>-Element aus der aktuellen Datei finden
                text_element = root.find(".//tei:text", namespaces)

                # Wenn ein <teiHeader>-Element vorhanden ist, füge das Dateinamen-Attribut hinzu
                if header_element is not None:
                    # Füge das Attribut 'filename' mit dem Dateinamen als Wert zum <teiHeader>-Element hinzu
                    header_element.set('filename', filename)

                    # Füge das <teiHeader>-Element zur kombinierten Datei hinzu
                    combined_root.append(header_element)

                # Wenn ein <text>-Element vorhanden ist, füge es ebenfalls hinzu
                if text_element is not None:

                # Füge das Attribut 'filename' mit dem Dateinamen als Wert zum <text>-Element hinzu
                    text_element.set('filename', filename)
                    
                    combined_root.append(text_element)

    # Die kombinierte TEI-Datei speichern
    combined_tree = etree.ElementTree(combined_root)
    with open(output_file, "wb") as f:
        # XML-Deklaration einmalig schreiben
        f.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
        # Die kombinierte Datei mit Einrückung und UTF-8-Kodierung speichern
        f.write(etree.tostring(combined_tree, pretty_print=True, encoding="UTF-8"))

# Parameter festlegen
input_folder = "/Users/Mustermann/Desktop/Datei"  # Pfad zum Ordner, in dem die TEI-Dateien liegen
output_file = "/Users/Mustermann/Desktop/Zusammengefügte-Datei.xml"  # Pfad und Name der Ausgabedatei
library_city = "Dresden, Sächsische Landesbibliothek - Staats- und Universitätsbibliothek"  # Gesuchte Bibliothek


# Die Funktion ausführen, um die TEI-Dateien zu filtern und zusammenzuführen
merge_tei_files(input_folder, output_file, library_city)

# Erfolgsnachricht ausgeben
print(f"Die TEI-Dateien wurden erfolgreich in '{output_file}' zusammengeführt.")


### Zusammengefügte XML Dateien abgleichen
Überprüfen, ob die Ansätze mit Python und XQuery übereinstimmen. Dafür werden die Dateinamen der Schlegel-Briefe aus der SLUB miteinander verglichen.

In [ ]:
import os
from lxml import etree

def extract_filenames(file_path):
    """Extrahiert die Dateinamen aus einer XML-Datei"""
    tree = etree.parse(file_path)
    filenames = [filename.text for filename in tree.xpath("//TEI")]
    return set(filenames)  # Set für schnellen Vergleich

def compare_files(file1, file2):
    """Vergleicht die Dateinamen in zwei XML-Dateien und gibt Unterschiede aus"""
    # Dateinamen aus den beiden XML-Dateien extrahieren
    filenames_file1 = extract_filenames(file1)
    filenames_file2 = extract_filenames(file2)

    # Finde die Dateinamen, die in Datei 1, aber nicht in Datei 2 sind
    only_in_file1 = filenames_file1 - filenames_file2
    # Finde die Dateinamen, die in Datei 2, aber nicht in Datei 1 sind
    only_in_file2 = filenames_file2 - filenames_file1

    # Ausgabe der Unterschiede
    if only_in_file1:
        print(f"Dateinamen in {file1}, aber nicht in {file2}:")
        for filename in only_in_file1:
            print(filename)

    if only_in_file2:
        print(f"Dateinamen in {file2}, aber nicht in {file1}:")
        for filename in only_in_file2:
            print(filename)
    else:
        print("Dateien stimmen überein")

# Pfade zu den beiden gemergten XML-Dateien
file1 = "/Users/Mustermann/Desktop/Zusammengefügte-Datei-1.xml"  # Pfad zur ersten XML-Datei
file2 = "/Users/Mustermann/Desktop/Zusammengefügte-Datei-2.xml"  # Pfad zur zweiten XML-Datei

# Vergleiche die Dateien und gib die Unterschiede aus
compare_files(file1, file2)


Dateien stimmen überein
